<a href="https://colab.research.google.com/github/1987Shiz321/BetterRecolor/blob/refactor%2Fseparated-notebook/ButtonTextReColorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Googleドライブのマウント**
Googleドライブにアクセスして、ファイルの読み書きなどを行うために必要なコードです。実行した後、アクセスを許可してください。\
なお、このノートブックでは指定されたディレクトリ以外のファイル(ご自身の写真やドキュメントデータなど)を読み書きすることはありませんので、ご安心ください。

In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 環境の準備
以下のセルを実行すると、必要なツールの導入などを行います。

**2025/11/23 0時ごろ(日本時間)にアップデートを行いました。ファイルの変更を行うために以下のコードを実行してください。**

In [ ]:
# Googleドライブのマイドライブへ移動.
%cd /content/drive/MyDrive
#マイドライブ直下に8MeToolsフォルダを生成.
!mkdir -p ./8MeTools/BTRC
#8MeToolsフォルダに移動
%cd /content/drive/MyDrive/8MeTools
#8MeToolsフォルダ内に必要ファイルをダウンロード.
!git clone https://github.com/8MeTools/ButtonTextReColorizer.git
#フォルダ名のリネーム.
!mv "/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/"* "/content/drive/MyDrive/8MeTools/BTRC/"
#不要データの削除(GitHub上では必要ですが、作業においては必要のないものです)
!rm -r "/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/"
!rm /content/drive/MyDrive/8MeTools/BTRC/EditedBRLAN/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/EditedBRLYT/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLAN/.gitkeep
!rm /content/drive/MyDrive/8MeTools/BTRC/tmp/BRLYT/.gitkeep
# ファイル,ディレクトリが存在しているかどうかを確認する
%ls /content/drive/MyDrive/8MeTools/BTRC -l

In [ ]:
%cd /content/drive/MyDrive/8MeTools/ButtonTextReColorizer
!git clone https://github.com/stblr/wuj5.git
!pip install colour colr
!pip install json5
!ls -l

In [ ]:
!pwd

/content/wuj5


In [3]:

import sys
ROOTPATH ='/content/drive/MyDrive/8MeTools/BetterRecolor/'
sys.path.append(ROOTPATH)

import btrc

In [19]:
from btrc.config import BRLYT_JSON5_DIR, BRLAN_JSON5_DIR, WUJ5_SCRIPT, EDITED_BRLYT_DIR, EDITED_BRLAN_DIR
from btrc.json5_io import list_json5_files, read_json5, write_json5
from btrc.brlyt import apply_tev_colors
from btrc.brlan import update_tev_colors, select_color_rule
from btrc.encode import encode_json5_files
from btrc.cleanup import remove_json5_files, move_all_files
from btrc.colors import run_color_input_flow, get_outline_color_from_user

In [20]:
print(BRLYT_JSON5_DIR, BRLAN_JSON5_DIR, WUJ5_SCRIPT, EDITED_BRLYT_DIR, EDITED_BRLAN_DIR)

/content/drive/MyDrive/8MeTools/ButtonTextReColorizer/tmp/BRLYT /content/drive/MyDrive/8MeTools/ButtonTextReColorizer/tmp/BRLAN /content/drive/MyDrive/8MeTools/ButtonTextReColorizer/wuj5/wuj5.py /content/drive/MyDrive/8MeTools/ButtonTextReColorizer/EditedBRLYT /content/drive/MyDrive/8MeTools/ButtonTextReColorizer/EditedBRLAN


In [7]:
color_map = run_color_input_flow()


=== fuchi_pattern2 のカラー設定 ===
デフォルト → Black: #ffffff, White: #000000
fuchi_pattern2のBlackColorを入力してください: #64C3C1
fuchi_pattern2のWhiteColorを入力してください:  #19BFB3

=== color_base2 のカラー設定 ===
デフォルト → Black: #434343, White: #434343
color_base2のBlackColorを入力してください:  #19BFB3
color_base2のWhiteColorを入力してください:  #19BFB3

=== color_yajirushi のカラー設定 ===
デフォルト → Black: #FFFFFF, White: #C8C8C8
color_yajirushiのBlackColorを入力してください: #FFFFFF
color_yajirushiのWhiteColorを入力してください: #19BFB3

=== ability_graph2 のカラー設定 ===
デフォルト → Black: #434343, White: #434343
ability_graph2のBlackColorを入力してください: #64C3C1
ability_graph2のWhiteColorを入力してください: #64C3C1

=== カラープレビュー ===
◆fuchi_pattern2 のカラー◆
BlackColor: #64C3C1 ■
WhiteColor: #19BFB3 ■
◆color_base2 のカラー◆
BlackColor: #19BFB3 ■
WhiteColor: #19BFB3 ■
◆black_base2 のカラー◆
BlackColor: #19BFB3 ■
WhiteColor: #19BFB3 ■
◆pikapika のカラー◆
BlackColor: #19BFB3 ■
WhiteColor: #19BFB3 ■
◆color_yajirushi のカラー◆
BlackColor: #FFFFFF ■
WhiteColor: #19BFB3 ■
◆ability_graph2 のカラー◆
BlackColor: 

In [8]:
default_free_text = (220, 220, 220)
default_select_text = (255, 255, 255)
free_outline = get_outline_color_from_user("free", "#282828")
select_outline = get_outline_color_from_user("select", "#787878")
text_free_colors = (default_free_text, free_outline)
text_select_colors = (default_select_text, select_outline)

free の縁取りカラー（空欄でデフォルト #282828）: #008a87
select の縁取りカラー（空欄でデフォルト #787878）: #1ddbd7


In [27]:
import os
import shutil

os.makedirs(BRLYT_JSON5_DIR, exist_ok=True)
os.makedirs(BRLAN_JSON5_DIR, exist_ok=True)

def copy_all(src_dir, dst_dir):
    for root, _, files in os.walk(src_dir):
        rel = os.path.relpath(root, src_dir)
        dst_root = os.path.join(dst_dir, rel) if rel != "." else dst_dir
        os.makedirs(dst_root, exist_ok=True)
        for f in files:
            src = os.path.join(root, f)
            dst = os.path.join(dst_root, f)
            shutil.copy2(src, dst)

copy_all(str(BRLYT_JSON5_DIR), str(BRLYT_JSON5_DIR))
copy_all(str(BRLAN_JSON5_DIR), str(BRLAN_JSON5_DIR))

None


In [23]:
brlyt_files = list_json5_files(BRLYT_JSON5_DIR)
for path in brlyt_files:
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    new_text = apply_tev_colors(text, color_map)
    if new_text is not None:
        with open(path, "w", encoding="utf-8") as f:
            f.write(new_text)

In [24]:
brlan_files = list_json5_files(BRLAN_JSON5_DIR)
for path in brlan_files:
    data = read_json5(path)
    rule = select_color_rule(path, text_free_colors, text_select_colors)
    if rule is None:
        continue
    (start_outline, start_text), (end_outline, end_text) = rule
    updated = update_tev_colors(data, start_outline, start_text, end_outline, end_text)
    write_json5(path, updated)

In [26]:
encode_json5_files(brlyt_files + brlan_files, WUJ5_SCRIPT)
remove_json5_files(brlyt_files + brlan_files)

NameError: name 't' is not defined

In [16]:
import os
os.makedirs(EDITED_BRLYT_DIR, exist_ok=True)
os.makedirs(EDITED_BRLAN_DIR, exist_ok=True)
move_all_files(BRLYT_JSON5_DIR, EDITED_BRLYT_DIR)
move_all_files(BRLAN_JSON5_DIR, EDITED_BRLAN_DIR)

Googleドライブを開いてファイルをダウンロードしてください。お疲れ様でした。